In [2]:
import json
import pandas as pd
import numpy as np



In [3]:
def load_json(file_path):
    """
    Load a JSON Lines file into a list of dictionaries.
    
    Args:
        file_path (str): Path to the JSON Lines file.
        
    Returns:
        list: A list of dictionaries representing the JSON objects in the file.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


data = load_json('data/football_lineups/extracted_player_json_files/extracted_match_data_385366.json')



In [20]:
def parse_lineups(match_json):
    """
    Parse the lineups from a match JSON object.
    
    Args:
        match_json (dict): The JSON object containing match data.
        
    Returns:
        pd.DataFrame: A DataFrame containing the parsed lineups.
    """
    lineups = []
    
    for team in match_json['lineups'].keys():
        starters = match_json['lineups'][team]['starters']
        substitutes = match_json['lineups'][team]['substitutes_played']

        lineups += starters
        lineups += substitutes
    
    return pd.DataFrame(lineups)

parse_lineups(data)

,number,name,id,sub_off_at,sub_on_at
0,1,D Raya,96467,None,NaN
1,7,Canós,104797,None,NaN
2,20,Ajer,92320,71'',NaN
3,18,Jansson,39717,None,NaN
4,5,Pinnock,120702,None,NaN
5,3,Henry,97186,None,NaN
6,15,Onyeka,107783,80'',NaN
7,6,Nørgaard,64477,None,NaN
8,27,Janelt,117505,None,NaN
9,19,Mbeumo,125509,86'',NaN
